In [0]:
from keras import Sequential
from keras.layers import Dense
from keras import regularizers
import pandas as pd
from random import randint
import numpy as np
from numpy.random import normal

#Utility Functions

In [0]:
def show_features(pop):
  features = []
  no_of_features = 0
  for i in range(len(pop)):

    if pop[i]==1:
      features.append(i)
      no_of_features += 1
  return features, no_of_features

In [0]:
def find_index_value(x,prob):
    for i in range(len(prob)):
        if x<prob[i]:
            return i
    return len(prob)-1

#Convert to binary encoding

In [0]:
def conver_bin(a):
    x = []
    while a>0:
        x.append(a%2)
        a //= 2
    x.reverse()
    return x

#Initialize population

In [0]:
def init_pop(number_bits, size):
    
    #taking samples over the distributtion
    #such that no. of 0 almost= no. of 1
    
    tot = int(2**number_bits)
    u = tot // size
    l = 1
    pop = []
    for i in range(size):
        p =[]
        num = randint(l,u)
        p = conver_bin(num)
        p = ([0]*(number_bits-len(p))) + p
        l = u+1
        u = (i+2) * (tot//size)
        pop.append(p)
    return pop

#Fitness Fuction (NN)

In [0]:
def fitness(chromo,x,y):
  
  chromo = np.asarray(chromo)
  x = np.multiply(x,chromo)
  #print(chromo.shape,x.shape)
    
  model = Sequential()

  model.add(Dense(10,activation='relu',activity_regularizer=regularizers.l2(0.01)))
  #model.add(Dense(10,activation='relu',activity_regularizer=regularizers.l2(0.01)))
  model.add(Dense(2,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy',
                optimizer='adam',metrics=['accuracy'])
  
  h= model.fit(x, y,
          batch_size=50,
          epochs=10,
          verbose=1)
  
  acc = h.history['acc'][-1]
  w_acc = 0.75
  w_f = 0.25
  value = acc #/ len(show_features(chromo)[0])
  
  return value

#Roulette Wheel Selection Technique

In [0]:
def roul(size, fit,pop,number_bits):
    tot = sum(fit)
    prob = [round((fit[0]/tot)*1000)]
    
    for i in range(1,size):
        x = fit[i]/tot
        x = round(x*1000)
        prob.append(prob[-1]+x)

    sel = []
    len_prob = len(prob)
    
    #using the *1000 and cumilative freq one
    for i in range(size//2):
        x = find_index_value(randint(1,999),prob)
        sel.append(pop[x])
        y = find_index_value(randint(1,999),prob)
        while x==y:
            y = find_index_value(randint(1,999),prob)
        sel.append(pop[y])
    
    return sel

#Crosser Over

In [0]:
def cross_over(selected, size, number_bits, cross_over_type='single'):
    if cross_over_type=='single':
        cross_over_sites = []
        for i in range(size//2):
            x = randint(1,number_bits-2)
            while x in cross_over_sites:
                x = randint(1,number_bits-2)
            cross_over_sites.append(x)

        pop = []

        for i in range(size//2):
            a = selected[i*2][:cross_over_sites[i]] + selected[i*2+1][cross_over_sites[i]:]
            b = selected[i*2+1][:cross_over_sites[i]] + selected[i*2][cross_over_sites[i]:]
            pop.append(a)
            pop.append(b)
            
    if cross_over_type=='uniform':
        cross_over_sites = []
        
        for i in range(size//2):
            temp_sites = []
            for j in range(randint(number_bits//2-2,number_bits//2+2)):
                temp_sites.append(randint(0,number_bits-1))
            cross_over_sites.append(temp_sites)
        
        pop = []
        
        for i in range(size//2):
            a = selected[i*2]
            b = selected[i*2+1]
            
            for j in cross_over_sites[i]:
                a[j], b[j] = b[j], a[j]
            
            pop.append(a)
            pop.append(b)
          
    return pop, cross_over_sites

#Mutation Using Normal Method|

In [0]:
def mutation(pop, mu_prob):
    tot = 0
    for i in pop:
        for j in range(len(i)):
            if normal(3,1) < mu_prob: #check this dis
                i[j] = (i[j]+1)%2
                tot += 1
    return pop, tot

#Running GA

In [0]:
# change this to its proper form
gen = 5
size = 6
number_bits = 29#do not make number_bits<size : screws up algo
mu_prob = 0.8
#mu_prob is not a prob but a value between 0 to 6 (normal dist graph with mean 3 and std 1)
pop = init_pop(number_bits, size)


In [0]:
features =[]
feature_fit = []

In [0]:
for i in range(gen):
    fit = []
    
    print('==============================================================================================')
    print('\n\nGen',i)
    
    
    #====================== eval fitness ======================
    for j in range(size):
        fit.append(fitness(pop[j],x_train,y_train))
        print('chromo:',j)
    
    avg, mx = sum(fit), max(fit)
    avg /= size
    best_in_gen = pop[fit.index(mx)]
    
    
    #====================== selecting the chormosomes using roulette wheel method ======================
    selected = roul(size,fit,pop, number_bits)
    
    #print('\nSelected pop',selected)
    
    
    #====================== crossover of chromosomes ======================
    pop, cross_over_sites = cross_over(selected,size,number_bits,cross_over_type='uniform')
    
    #print('\nNew pop', pop)
    #print('\nCross over sites', cross_over_sites)
    
    
    #============== mutation ====================================
    pop,tot_mu = mutation(pop,mu_prob)
    #print('\nAfter mutation', pop)
    #print('\nTotal mutations', tot_mu)
    
    
    #====================== analysis of new generation ======================
   
    
    print('\nMax : {}\nAvg : {}'.format(mx,avg))
    _,num_features = show_features(pop)
    print('Number of features of best in gen: ',num_features)
    features = pop
    feature_fit = fit

In [0]:
for j in range(size):
  print(fitness(features[j],x_train,y_train))

In [0]:
f,num_features = show_features(features[feature_fit.index(max(feature_fit))])
fitness(features[feature_fit.index(max(feature_fit))],x_train,y_train)
print(f)
print(num_features)